<a href="https://colab.research.google.com/github/Satyake/Poem-Generation/blob/main/Poetry_Generation_Laurence_Mooneys_Poem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec 
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
with open('/content/laurencepoem.txt') as f:
  file=f.read()

In [3]:
len(file)

68969

In [189]:
#file=file.split('\n')

In [4]:
file

'Come all ye maidens young and fair\nAnd you that are blooming in your prime\nAlways beware and keep your garden fair\nLet no man steal away your thyme\nFor thyme it is a precious thing\nAnd thyme brings all things to my mind\nnlyme with all its flavours, along with all its joys\nThyme, brings all things to my mind\nOnce I and a bunch of thyme\ni thought it never would decay\nThen came a lusty sailor\nWho chanced to pass my way\nAnd stole my bunch of thyme away\nThe sailor gave to me a rose\nA rose that never would decay\nHe gave it to me to keep me reminded\nOf when he stole my thyme away\nSleep, my child, and peace attend thee\nAll through the night\nGuardian angels God will send thee\nSoft the drowsy hours are creeping\nHill and dale in slumber sleeping\nI my loving vigil keeping\nWhile the moon her watch is keeping\nWhile the weary world is sleeping\nOer thy spirit gently stealing\nVisions of delight revealing\nBreathes a pure and holy feeling\nThough I roam a minstrel lonely\nMy t

In [5]:
import spacy
nlp=spacy.load('en')


In [ ]:
#stopwords.words('english')

In [23]:
#file=nlp('/content/laurencepoem.txt')

In [6]:
def seperate(doc):
  return [token.text for token in nlp(doc) if token.text not in ' \n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ' ]  


In [7]:
words=seperate(file)

In [8]:
train_length=30
processed_text_sequence=[]
for i in range(train_length,len(words)):
  seq=words[i-30:i]
  processed_text_sequence.append(seq)

In [9]:
' '.join(processed_text_sequence[0])

'Come all ye maidens young and fair And you that are blooming in your prime Always beware and keep your garden fair Let no man steal away your thyme For'

In [10]:
import tensorflow as tf
import numpy as np 

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [12]:
Tknizer=Tokenizer(oov_token='<OOV>')


In [13]:
Tknizer.fit_on_texts(processed_text_sequence)
sequences=Tknizer.texts_to_sequences(processed_text_sequence)

In [14]:
sequences=np.array(sequences)

In [15]:
sequences.shape

(13893, 30)

In [16]:
sequences

array([[  56,   13,  107, ...,   25,  233,   12],
       [  13,  107, 2669, ...,  233,   12,  233],
       [ 107, 2669,   53, ...,   12,  233,   26],
       ...,
       [ 272, 2667,  361, ...,  157,  357,   49],
       [2667,  361, 1200, ...,  357,   49,  110],
       [ 361, 1200, 1201, ...,   49,  110,  156]])

In [21]:
y=sequences[:,-1]
x=sequences[:,:-1]

In [34]:
y=tf.keras.utils.to_categorical(y,num_classes=vocab_size+1)

In [23]:
y.shape

(13893,)

In [53]:
seq_length=x.shape[0]
print(seq_length)

13893


In [54]:
from keras.models import Sequential

from keras.layers import Dense,LSTM,Embedding,Dropout,Bidirectional
import tensorflow as tf
def model1(vocab_size,seq_length):
  model=Sequential()
  model.add(Embedding(vocab_size+1,seq_length,input_length=seq_length))
  model.add(Bidirectional(LSTM(300,activation='relu',return_sequences=True)))
  model.add(Bidirectional(LSTM(300,activation='relu',return_sequences=False)))
  model.add(Dropout(0.5))
  model.add(Dense(300,activation='relu'))
  model.add(Dense(2669,activation='softmax'))
  model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
  return model

In [55]:
vocab_size=len(Tknizer.word_counts)
model=model1(vocab_size+1,seq_length)

In [56]:
patience = 1
stop_patience = 3
factor = 0.5

callbacks = [
    #tf.keras.callbacks.ModelCheckpoint("classify_model.h5", save_best_only=True, verbose = 0),
    tf.keras.callbacks.EarlyStopping(patience=stop_patience, monitor='loss', verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=factor, patience=patience, verbose=1)
]

In [ ]:
history=model.fit(x,y,epochs=110,batch_size=128)

In [119]:
#padded_sentencces=pad_sequences(sequences,maxlen=max_len)

In [58]:
model.save('GoodLSTMModel.h5')

In [59]:
def generate_text(model,tokenizer,seq_len,seed_text,num_gen_words):
  output_text=[]
  input_text=seed_text
  for i in range(num_gen_words):
    encoded=tokenizer.texts_to_sequences([input_text])[0]
    pad_encoded=pad_sequences([encoded],maxlen=seq_len)
    pred_word_index=model.predict_classes(pad_encoded)[0]
    pred_word=tokenizer.index_word[pred_word_index]
    input_text+=' '+pred_word
    output_text.append(pred_word)
  return ' '.join(output_text)

In [60]:
seed_text='I just met you'
seed_text

'I just met you'

In [61]:
generate_text(model,Tknizer,seq_length,seed_text,num_gen_words=25)

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


'a wobblin of thyme he s as rowans are delight in thundering mountain side by on fire and lasses to your wedding she s no'